In [ ]:
import pandas as pd
df = pd.read_csv('C:/Users/dstoc/Documents/Python Scripts/Fixed Income Dashboards/reduced_data_cleaned.csv')

In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from math import sqrt


df2 = df.copy()

# Assuming 'Date' is the name of the column containing date information
df2['Date'] = pd.to_datetime(df2['Date'])
df2.set_index('Date', inplace=True)
df2 = df2.replace([np.inf, -np.inf], np.nan).dropna(axis = 1)

# Define the features (independent variables) and target (dependent variable)
features = list(df2)
# Define the lag order (number of lags to create)
lag_order = 6  # Example: Creating lagged values for the past n periods
# Define the initial training size (e.g., 70% of the data)
initial_train_size = int(0.7 * len(df2)) - lag_order
InSamp_length = len(df2) - initial_train_size - lag_order


In [ ]:

# Define the number of periods to lead the target variable (1 period in this case)
lead_periods = 1

# redefine the features (independent variables) and target (dependent variable)
features = list(df2)

# Create a new variable representing the 'lead' of the target
df2['YIELD_weekly_percent_change_10 YR_lead'] = df2['YIELD_weekly_percent_change_10 YR'].shift(-lead_periods)

# Drop rows with missing values in the features or target (created by the shift)
df2 = df2.dropna(subset=features + ['YIELD_weekly_percent_change_10 YR_lead'])

In [ ]:
df2['YIELD_weekly_percent_change_10 YR'].tail()

In [ ]:
df3 = df2.copy()
features = list(df3.select_dtypes(include=['float64', 'int64']).columns)[0:(len(df3.columns)-1)]
X = df3[features]
y = df3['YIELD_weekly_percent_change_10 YR_lead']


In [ ]:
df3 = df2.copy()
features = list(df3.columns)[0:(len(df3.columns)-1)]
X = df3[features]
y = df3['YIELD_weekly_percent_change_10 YR_lead']

target_var = 'YIELD_weekly_percent_change_10 YR_lead'

if target_var.lower() in map(str.lower, X.columns):
    print(f"The DataFrame contains the column '{target_var}'.")
else:
    print(f"The DataFrame X does not contain the target variable: '{target_var}'.")

Spearman's rank correlation is often used when the variables being compared may not have a linear relationship or when the assumptions of parametric correlation measures like Pearson's correlation are not met.

In [ ]:
df3.iloc[:InSamp_length,].tail()

In [ ]:
import pandas as pd

def select_top_variables(df, target_variable, num_variables):
    # Calculate the rank correlation (Spearman's rho) with the target variable
    correlation_matrix = df.corrwith(df[target_variable], method='spearman')
    
    # Sort variables based on absolute correlation values
    sorted_variables = correlation_matrix.abs().sort_values(ascending=False)
    
    # Select the top variables
    selected_variables = sorted_variables.index[:num_variables]
    
    # Extract the correlation values for the selected variables
    selected_correlations = correlation_matrix[selected_variables]
    
    # Create a DataFrame to store the results
    corr_data = pd.DataFrame({
        'Variable': selected_variables,
        'Correlation': selected_correlations
    })
    
    return corr_data

# Assuming df3 is your DataFrame and 'YIELD_weekly_percent_change_10 YR' is the target variable
correlated_variables = select_top_variables(df3.loc[df3.index[:InSamp_length]], 'YIELD_weekly_percent_change_10 YR_lead', 200)


In [ ]:
df3.loc[df3.index[:InSamp_length], correlated_variables['Variable']].head()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# Assuming X is your DataFrame containing the selected variables
#vif_results = calculate_vif(df3.iloc[:InSamp_length,[correlated_variables['Variable']]])
vif_results = calculate_vif(df3.loc[df3.index[:InSamp_length], correlated_variables['Variable']])

# Print the variables with high VIF
low_vif_variables = vif_results.sort_values('VIF')[:51]
print("Variables with low VIF:")
print(low_vif_variables)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, make_scorer

# Assuming df4 is your DataFrame
df6 = df3.copy()
df6 = df6[low_vif_variables["Variable"]]
# Define your features and target variable
features = low_vif_variables["Variable"]
features2 = [feature for feature in features if feature != target_var]


for feature in features:
    for lag in range(1, lag_order+1):
        df6[f'{feature}_lag{lag}'] = df6[feature].shift(lag)


# replace inf and -inf with na and  drops rows with missing values in the lagged variables
df6 = df6[6:]
df6 = df6.replace([np.inf, -np.inf], np.nan).dropna(axis = 1)

In [ ]:
df6[target_var] = np.where(df6[target_var] < 0, 'LONG', 'SHORT')
X = df6[features2]
y = df6[target_var]

if target_var.lower() in map(str.lower, X.columns):
    print(f"The DataFrame contains the column '{target_var}'. ABORT BACK-TEST")
else:
    print(f"The DataFrame X does not contain the target variable: '{target_var}'. You may proceed with the back-test.")


                                                                    
                                                                    ## Gridsearch performed below ##

In [ ]:
# Set up the parameter grid for grid search
# param_grid = {
#     'n_estimators': [100, 150],
#     'learning_rate': [0.01, 0.1],
#     'subsample': [0.75],
#     'max_depth': [3, 13],
#     'min_samples_leaf': [2, 5]
# }

param_grid = {
    'n_estimators': [50],
    'learning_rate': [0.3],
    'subsample': [0.75],
    'max_depth': [5,15],
    'min_samples_leaf': [5]
}

# Initialize the Gradient Boosting Classifier
gbm = GradientBoostingClassifier(random_state=248)


ys = []

def Acc_Score(y_true,y_pred):
    global ys
    ys.append(y_pred)
    acc = accuracy_score(y_true, y_pred)    
    return acc

def scorer():
    return make_scorer(Acc_Score, greater_is_better=True)

# Create the time series split for cross-validation
tscv = TimeSeriesSplit(n_splits=len(df6) - initial_train_size, test_size=1,gap=0, max_train_size=None)

# Perform grid search with cross-validation
grid_search = GridSearchCV(gbm, param_grid, cv=tscv, scoring=scorer(), verbose=1)#, n_jobs=-1)
grid_search.fit(X, y)

# Get the best model from the grid search
best_gbm = grid_search.best_estimator_

cv_results = pd.DataFrame(grid_search.cv_results_)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)
print(f'Accuracy of the Best Hyperparameters: {grid_search.best_score_:.4f}')


In [ ]:
ys

In [ ]:
from sklearn.metrics import confusion_matrix 
#Calculate and display accuracy
accuracy_score(y[(initial_train_size):], ys[(((len(df6) - initial_train_size) * (grid_search.best_index_))):(len(df6) - initial_train_size) * (grid_search.best_index_ +1)])

In [ ]:
import joblib
# save the model to disk
filename = '10Y_UST_gridsearch_results.sav'
joblib.dump(grid_search, filename)

In [ ]:
best_estim_preds = ys[(((len(df6) - initial_train_size) * (grid_search.best_index_))):(len(df6) - initial_train_size) * (grid_search.best_index_ +1)]

In [ ]:
best_estim_preds = np.array(best_estim_preds).flatten().tolist()


In [ ]:
joblib.dump(best_estim_preds,'10Y_UST_best_estimator_predictions.sav')

In [ ]:
class backtest_object:
  def __init__(self,grid_search, X, y, best_estim_preds, final_model):
    self.grid_search = grid_search
    self.X = X
    self.y = y
    self.best_estim_preds = best_estim_preds
    self.final_model = best_gbm

backtest_object_10Y_UST_v1_1 = backtest_object(grid_search, X, y, best_estim_preds, best_gbm)
print(backtest_object_10Y_UST_v1_1)

import pickle

# Serialize the object using pickle and save to a file
with open('backtest_object_10Y_UST_v1_1.pkl', 'wb') as file:
    pickle.dump(backtest_object_10Y_UST_v1_1, file)
    


In [ ]:
y.tail()

code for predicting off of new data

In [ ]:
grid_search.best_estimator_.predict(X.tail(1))

In [ ]:
grid_search.best_estimator_.predict_proba(X.tail(1))

In [ ]:
# https://machinelearningmastery.com/modeling-pipeline-optimization-with-scikit-learn/
import seaborn as sns 
sns.relplot(data=cv_results,
 kind='line',
 x='param_subsample',
 y='mean_test_score',
 hue='param_learning_rate',
 col='param_max_depth')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.hist(cv_results['mean_test_score'])
plt.xlabel('% Accuracy')
plt.ylabel('Count')
 
plt.title('Distribution of Out-of-Sample Accuracy Scores of Hyperparameter Combos\n\n',
          fontweight = "bold")

In [ ]:
import matplotlib.pyplot as plt
group1 = cv_results.query('param_learning_rate == 0.3')
group2 = cv_results.query('param_learning_rate == 0.1')

plt.hist(group1['mean_test_score'], label='LR = 0.3', alpha=0.75)
plt.hist(group2['mean_test_score'], label='LR = 0.1', alpha=0.75)
plt.xlabel('% Accuracy')
plt.ylabel('Count')
plt.legend(loc='upper right')
plt.title('Distribution of Out-of-Sample Accuracy Scores of Hyperparameter Combos\n\n',
          fontweight = "bold")

In [ ]:
import matplotlib.pyplot as plt
group1 = cv_results.query('param_max_depth == 15')
group2 = cv_results.query('param_max_depth == 5')

plt.hist(group1['mean_test_score'], label='Max Depth = 15', alpha=0.75)
plt.hist(group2['mean_test_score'], label='Max Depth = 5', alpha=0.75)
plt.xlabel('% Accuracy')
plt.ylabel('Count')
plt.legend(loc='upper right')
plt.title('Distribution of Out-of-Sample Accuracy Scores of Hyperparameter Combos\n\n',
          fontweight = "bold")

In [ ]:
import matplotlib.pyplot as plt
group1 = cv_results.query('param_min_samples_leaf == 5')
group2 = cv_results.query('param_min_samples_leaf == 1')

plt.hist(group1['mean_test_score'], label='Min Samples Leaf = 8', alpha=0.75)
plt.hist(group2['mean_test_score'], label='Min Samples Leaf = 1', alpha=0.75)
plt.xlabel('% Accuracy')
plt.ylabel('Count')
plt.legend(loc='upper right')
plt.title('Distribution of Out-of-Sample Accuracy Scores of Hyperparameter Combos\n\n',
          fontweight = "bold")

In [ ]:
import matplotlib.pyplot as plt
group1 = cv_results.query('param_n_estimators == 50')
group2 = cv_results.query('param_n_estimators == 100')

plt.hist(group1['mean_test_score'], label='n_estimators = 50', alpha=0.75)
plt.hist(group2['mean_test_score'], label='n_estimators = 100', alpha=0.75)
plt.xlabel('% Accuracy')
plt.ylabel('Count')
plt.legend(loc='upper right')
plt.title('Distribution of Out-of-Sample Accuracy Scores of Hyperparameter Combos\n\n',
          fontweight = "bold")

In [ ]:
import matplotlib.pyplot as plt
group1 = cv_results.query('param_subsample == 1')
group2 = cv_results.query('param_subsample == .75')

plt.hist(group1['mean_test_score'], label='subsample = 1.0', alpha=0.75)
plt.hist(group2['mean_test_score'], label='subsample = 0.75', alpha=0.75)
plt.xlabel('% Accuracy')
plt.ylabel('Count')
plt.legend(loc='upper right')
plt.title('Distribution of Out-of-Sample Accuracy Scores of Hyperparameter Combos\n\n',
          fontweight = "bold")

In [ ]:
tscv.split(X)

In [ ]:
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
X_train.index


In [ ]:
X_test.index

In [ ]:
X.index[(len(df6) - initial_train_size):]

In [ ]:
# plot your own learning curve

cv_perf = pd.DataFrame(cv_results.iloc[0,10:282])
cv_perf = cv_perf.rename(columns={0: 'score'})
cv_perf['Expanding_Average'] = cv_perf['score'].expanding().mean()
#cv_perf.reindex(X.index[(len(df6) - initial_train_size):])
# Plot the expanding average
plt.figure(figsize=(10, 6))
#plt.plot(cv_perf['Value'], label='Original Values', marker='o')
plt.plot(cv_perf['Expanding_Average'], label='Expanding Average', linestyle='--', marker='o')
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Cross Validation Scoring Plot')
plt.legend()
# Show only every 10th tick on the x-axis
plt.xticks(cv_perf.index[::50])
plt.show()


In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.model_selection import learning_curve, train_test_split

# # Get the best estimator from the grid search
# best_gbm = grid_search.best_estimator_

# # Plot learning curve
# train_sizes, train_scores, test_scores = learning_curve(best_gbm, X_train, y_train, cv=tscv, scoring=make_scorer(accuracy_score), train_sizes=np.ones((len(df6) - initial_train_size)))

# # Calculate mean and standard deviation for training set scores and test set scores
# train_mean = np.mean(train_scores, axis=1)
# train_std = np.std(train_scores, axis=1)
# test_mean = np.mean(test_scores, axis=1)
# test_std = np.std(test_scores, axis=1)

# # Plot the learning curve
# plt.figure(figsize=(10, 6))
# plt.plot(train_sizes, train_mean, label='Training accuracy', color='blue', marker='o')
# plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.2)
# plt.plot(train_sizes, test_mean, label='Cross-validation accuracy', color='red', marker='o')
# plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color='red', alpha=0.2)

# # Add labels and legend
# plt.title('Learning Curve for GBM Classifier')
# plt.xlabel('Number of training examples')
# plt.ylabel('Accuracy')
# plt.legend(loc='best')

# # Show the plot
# plt.show()


In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.model_selection import learning_curve, train_test_split

# # Get the best estimator from the grid search
# best_gbm = grid_search.best_estimator_

# # Plot learning curve
# train_sizes, train_scores, test_scores = learning_curve(best_gbm, X_train, y_train, cv=tscv, scoring=make_scorer(accuracy_score), train_sizes=np.ones((len(df6) - initial_train_size)))

# # Calculate mean and standard deviation for training set scores and test set scores
# train_mean = np.mean(train_scores, axis=1)
# train_std = np.std(train_scores, axis=1)
# test_mean = np.mean(test_scores, axis=1)
# test_std = np.std(test_scores, axis=1)

# # Plot the learning curve
# plt.figure(figsize=(10, 6))
# plt.plot(train_sizes, train_mean, label='Training accuracy', color='blue', marker='o')
# plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.2)
# plt.plot(train_sizes, test_mean, label='Cross-validation accuracy', color='red', marker='o')
# plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color='red', alpha=0.2)

# # Add labels and legend
# plt.title('Learning Curve for GBM Classifier')
# plt.xlabel('Number of training examples')
# plt.ylabel('Accuracy')
# plt.legend(loc='best')

# # Show the plot
# plt.show()
